In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_dataset = load_dataset("glue" , "mrpc")
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["sentence1"] , examples["sentence2"] , truncation= True) # We don't use paddint = True parameter this time.

tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [2]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], batch_size=8, shuffle=True, collate_fn=data_collator
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, shuffle=True, collate_fn=data_collator
)

In [3]:
# We grab a batch of data for looked it.
for batch in train_dataloader:
    break

print({k: v.shape for k, v in batch.items()})

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 71]), 'token_type_ids': torch.Size([8, 71]), 'attention_mask': torch.Size([8, 71])}


In [4]:
from transformers import AutoModelForSequenceClassification

checkpoint = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

c:\Users\Acer\miniconda3\envs\nlp\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Acer\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['

In [5]:
# We try on a batch which we grab.
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.7042, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [6]:
from transformers import AdamW

optimizer = AdamW(model.parameters() , lr= 5e-5)

c:\Users\Acer\miniconda3\envs\nlp\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
loss = outputs.loss
loss.backward()
optimizer.step()

# Don't forget to zero your gradients! once your optimizer step is done!
optimizer.zero_grad()

In [8]:
from transformers import get_scheduler

num_epochs = 3
num_trainings_steps = len(train_dataloader) * num_epochs
lr_scheduler = get_scheduler(
    "linear",
    optimizer= optimizer,
    num_warmup_steps=0,
    num_training_steps= num_trainings_steps
    )

In [9]:
import torch

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)
# print(device)

# If you want to use Cuda GPU, you should transfer model and optimizer to GPU.
if torch.cuda.is_available():
    model.to('cuda')
    optimizer = AdamW(model.parameters(), lr=5e-5)  # Don't forget to re define optimizer after cuda.
    device = 'cuda'

In [14]:
# from tqdm.auto import tqdm
from accelerate import Accelerator

accelerator = Accelerator()

# progress_bar = tqdm(range(num_trainings_steps))

model, optimizer, train_dataloader = accelerator.prepare(model, optimizer, train_dataloader)

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        # loss.backward()
        accelerator.backward(loss)
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        # progress_bar.update(1)

In [20]:
from datasets import load_metric

metric = load_metric("glue" , "mrpc")
model.eval()

eval_dataloader = accelerator.prepare(eval_dataloader)
for batch in eval_dataloader:
    # batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    # metric.add_batch(predictions=predictions, references=batch["labels"])
    metric.add_batch(
        predictions= accelerator.gather(predictions) , references = accelerator.gather(batch['labels'])
    )

metric.compute()

{'accuracy': 0.8455882352941176, 'f1': 0.8919382504288165}

In [ ]:
# from accelerate import notebook_launcher

# notebook_launcher(my_training_function) # We need a training function which I don't know how to create yet.

# Or we can create our .py script to use it in terminal.

In [ ]:
# Done.